# CTIPe


The kamodofied CTIPe requires `netcdf4` (and numpy, scipy, and pandas, but those are required by kamodo)

    pip install netcdf4

!!! warning
    This is a work in progress!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from netCDF4 import Dataset
import numpy as np
import os
import scipy
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
import math
import time
import kamodo
from datetime import datetime,timezone

In [3]:
filename = '/Users/lrastaet/Kamodo_data/CTIPe/2020-05-04-plot-density.nc' # each CTIPe run has plot-density, plot-height, plot-neutral and plot-plasma files for each date
filename = '/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-density.nc' 

In [55]:
from readers.ctipe import CTIPe_Kamodo
from readers.geopack_transform import seconds_from_19700101
def seconds_from_19700101_to_DateTime(seconds):
    from datetime import datetime,timezone,timedelta
    time0=datetime(1970,1,1,0,0,0,tzinfo=timezone.utc)+timedelta(seconds=seconds)
    return time0
from tiegcm.util import datetime_to_epoch

By default, the kamodofied CTIPe reader will load all 3d and 4d variables from the model. For performance reasons, you may want to only include certain variables for analysis. To do so, pass a list of variables in addition to the filename:

In [56]:
ctipe = CTIPe_Kamodo(filename,debug=False)
#ctipe = CTIPe_Kamodo(filename, ['TN', 'UN', 'VN', 'EFLUX', 'Z'])

/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-density.nc
/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-height.nc
/Users/lrastaet/Kamodo_data/CTIPe/Sujay_Pal_042620_IT_1/2015-03-20-plot-neutral.nc
time
plev
lat
lon
doy
min
density
height
temperature
mean_molecular_mass
time
ht
lat
lon
electron_density
atomic_oxygen_ion_density
atomic_hydrogen_ion_density
ion_temperature
electron_temperature
time
plev
lat
lon
height
elat
elon
meridional_neutral_wind
zonal_neutral_wind
vertical_neutral_wind
neutral_temperature
mean_molecular_mass
electron_density
neutral_density
solar_heating
joule_heating
radiation_heat_cool
atomic_oxygen_density
molecular_oxygen_density
molecular_nitrogen_density
nitric_oxide_density
nitric_oxide_ion_density
molecular_nitrogen_ion_density
molecular_oxygen_ion_density
atomic_nitrogen_ion_density
atomic_oxygen_ion_density
atomic_hydrogen_ion_density
pedersen_conductivity
hall_conductivity
meridional_ion_velocity
zonal_io

In [57]:
ctipe

CTIPe_Kamodo([(rho(t, ilev, lat, lon),
               <function wrapped(t=array([1.4268105e+09, 1.4268114e+09, 1.4268123e+09, 1.4268132e+09,
       1.4268141e+09, 1.4268150e+09, 1.4268159e+09, 1.4268168e+09,
       1.4268177e+09, 1.4268186e+09, 1.4268195e+09, 1.4268204e+09,
       1.4268213e+09, 1.4268222e+09, 1.4268231e+09, 1.4268240e+09,
       1.4268249e+09, 1.4268258e+09, 1.4268267e+09, 1.4268276e+09,
       1.4268285e+09, 1.4268294e+09, 1.4268303e+09, 1.4268312e+09,
       1.4268321e+09, 1.4268330e+09, 1.4268339e+09, 1.4268348e+09,
       1.4268357e+09, 1.4268366e+09, 1.4268375e+09, 1.4268384e+09,
       1.4268393e+09, 1.4268402e+09, 1.4268411e+09, 1.4268420e+09,
       1.4268429e+09, 1.4268438e+09, 1.4268447e+09, 1.4268456e+09,
       1.4268465e+09, 1.4268474e+09, 1.4268483e+09, 1.4268492e+09,
       1.4268501e+09, 1.4268510e+09, 1.4268519e+09, 1.4268528e+09,
       1.4268537e+09, 1.4268546e+09, 1.4268555e+09, 1.4268564e+09,
       1.4268573e+09, 1.4268582e+09, 1.4268591e+09, 1.4

In [61]:
ctipe.lev_density(ctipe,positions[0],125000.,positions[2],positions[3])

array(7.01841074)

In [68]:
np.squeeze(ctipe.H(t=positions[0],lon=positions[2],lat=positions[3]))

array([ 77503.24913194,  83067.05208333,  88529.90625   ,  93959.75      ,
       100936.42361111, 111157.87934028, 124689.31423611, 141564.55555556,
       163143.01388889, 192617.29513889, 226364.76041667, 263201.68402778,
       302007.36458333, 341841.77430556, 382169.63888889])

In [ ]:
## comparison test with IDL in run 
## IDL> print,get_pointdata([50.,0.,10.])
## % Compiled module: GET_POINTDATA_3D.
##  5.07806e-10      188.940      19.6322     -6.30557      28.9232      1.83336      595.492  7.52705e+10  1.99925e-05   0.00973094
##    -0.985651  1.08981e+16  1.08948e+14  4.56949e+15  1.68988e+12  1.35428e+10      6169.57  7.52248e+09      29051.7  5.42052e+10
##  1.73583e+06  3.07283e-05  1.89448e-06     -3.32263      2.88644  1.39743e-06      0.00000      0.00000 -0.000311317 -0.000303866
## -0.000346761 -0.000338463      36.2960      0.00000  1.38551e+12      260.000
##IDL> print,qusntities
##% PRINT: Variable is undefined: QUSNTITIES.
##% Execution halted at: $MAIN$              1 /ccmcshare/IDL/3DView/ak3me.pro
##IDL> print,quantities
##rho H Rmt Vn_lat Vn_lon Vn_IP T_n N_e Psolar Pjoule Prad N_O N_O2 N_N2 N_NO N_NO+ N_N2+ N_O2+ N_N+ N_O+ N_H+ sigma_P sigma_H Vi_lat
##Vi_lon Wjoule Win En_avg E140_theta E300_theta E140_lambda E300_lambda TEC_above_140km Vi_IP NmF2 HmF2
##IDL> print,year,month,time,put_time(time)
##        2015           3   9.9999999e-0900:00:00.0000

In [102]:
ctipe.time2datetime(ctipe._time)

AttributeError: No such symbol: _Time

In [15]:
ctipe._time

array([1.4268105e+09, 1.4268114e+09, 1.4268123e+09, 1.4268132e+09,
       1.4268141e+09, 1.4268150e+09, 1.4268159e+09, 1.4268168e+09,
       1.4268177e+09, 1.4268186e+09, 1.4268195e+09, 1.4268204e+09,
       1.4268213e+09, 1.4268222e+09, 1.4268231e+09, 1.4268240e+09,
       1.4268249e+09, 1.4268258e+09, 1.4268267e+09, 1.4268276e+09,
       1.4268285e+09, 1.4268294e+09, 1.4268303e+09, 1.4268312e+09,
       1.4268321e+09, 1.4268330e+09, 1.4268339e+09, 1.4268348e+09,
       1.4268357e+09, 1.4268366e+09, 1.4268375e+09, 1.4268384e+09,
       1.4268393e+09, 1.4268402e+09, 1.4268411e+09, 1.4268420e+09,
       1.4268429e+09, 1.4268438e+09, 1.4268447e+09, 1.4268456e+09,
       1.4268465e+09, 1.4268474e+09, 1.4268483e+09, 1.4268492e+09,
       1.4268501e+09, 1.4268510e+09, 1.4268519e+09, 1.4268528e+09,
       1.4268537e+09, 1.4268546e+09, 1.4268555e+09, 1.4268564e+09,
       1.4268573e+09, 1.4268582e+09, 1.4268591e+09, 1.4268600e+09,
       1.4268609e+09, 1.4268618e+09, 1.4268627e+09, 1.4268636e

In [ ]:
import numpy as np
z = np.squeeze(ctipe.H(ctipe._time[0], lat = 0, lon = 0))

In [ ]:
ctipe['RHO(t, z, lat, lon)'] = 'rho(t,ilev, lat, lon)'

In [119]:
testdata=dict()
testdata['ip']=5.5
testdata['Height']='125'
testdata['date']='2015/3/20'
testdata['time']='00:15:00'
testdata['lon']=50.
testdata['lat']=10.
testdata['rho']='1.49067e-07  1.64051e-08'
testdata['H']='105.230      125.000'
testdata['Rmt']='26.3395      23.9059'
testdata['Vn_lat']='-0.0601084      43.4968'
testdata['Vn_lon']='-23.2656     -3.15513'
testdata['Vn_IP']='-0.0434272   -0.0818311'
testdata['T']='298.432      430.817'  #T_n
testdata['N_e']='1.06982e+08  5.90486e+08'
testdata['Psolar']='1.10576e-06  2.72283e-06'
testdata['Pjoule']='4.78753e-05  0.000398958'
testdata['Prad']=' -0.210089     -1.06214'
testdata['N_O']='4.70951e+17  1.42260e+17'
testdata['N_O2']='3.72280e+17  1.27961e+16'
testdata['N_N2']='2.51143e+18  2.56911e+17'
testdata['N_NO']='6.02975e+13  3.22197e+13'
testdata['N_NOplus']='1.06908e+08  5.04418e+08'
testdata['N_N2plus']='4.83005     0.987384'
testdata['N_O2plus']='74194.3  8.11062e+07'
testdata['N_Nplus']='0.615991     0.795714'
testdata['N_Oplus']='3.41415  4.96166e+06' #N_O+
testdata['N_Hplus']='0.00000      32.4158'
testdata['sigma_P']='9.66247e-07  5.87935e-06'
testdata['sigma_H']='9.17236e-06  8.91819e-06'
testdata['Vi_lat']='-0.723088      40.6683'
testdata['Vi_lon']='-23.9126     -9.70373'
testdata['Wjoule']='5.02903e-07  5.02903e-07'
testdata['Win ']='0.00000      0.00000'
testdata['En_avg']='0.00000      0.00000'
testdata['E_theta140km']='-0.000191656 -0.000191656'
testdata['E_theta300km']='-0.000187069 -0.000187069'
testdata['E_lambda140km']='-0.000340932 -0.000340932'
testdata['E_lambda300km']='-0.000332773 -0.000332773'
testdata['TEC']='40.6796      40.6796'  # 'TEC_above_140km'
testdata['Vi_IP']='0.00000      0.00000'
testdata['NmF2']='1.57999e+12  1.57999e+12'
testdata['HmF2']='300.000      300.000'

In [124]:
checked=dict()
pos_ip=[ctipe._time[0],5.5,10,50]
# the Python algortihm calculates IP and uses that for regular interpolation in grid
# resulting in a slightly smaller height value thatn input.
# the required input height value resulting in 125 km in the output is 126.2 km. 
# THis shift in altitude / height may vary over the longitude-latitude grid.
# the IDL implemetnation uses the height value and interpolates at the 2 nearest (lon,lat) 
# grid stacks using the height values before applying the weights in longitude and latitude 
# to arrive at the final interpolatied value for each quantity
# numerical value for efield parameters are different between IDL and Python
# The efield values on the 37x73 grid are re-interpolated to the 91x20 plasma grid, a step not performed in Python
#
pos_h=[ctipe._time[0],126200.,10,50]
#pos_h=[ctipe._time[0],125000.,10,50]
print(pos_ip)
print(pos_h)
ilev=ctipe.lev_density(ctipe,t=pos_h[0],z=pos_h[1],lon=pos_h[2],lat=pos_h[3])
pos_h_ip=pos_h
pos_h_ip[1]=ilev
print(pos_h_ip)
for var in ctipe:
    if var.name not in checked:  # prevent duplication
        checked[var.name]=1   #
        
        try:
            val_ip=np.squeeze(np.array(ctipe[var](pos_ip[0],pos_ip[1],pos_ip[2],pos_ip[3])))
        except:
            val_ip=np.squeeze(np.array(ctipe[var](pos_ip[0],pos_ip[2],pos_ip[3])))
        try:
            val_h=np.squeeze(np.array(ctipe[var](pos_h[0],ilev,pos_h[2],pos_h[3])))
        except:
            val_h=np.squeeze(np.array(ctipe[var](pos_h[0],pos_h[2],pos_h[3])))
        if var.name in testdata:
            print("%s %12.4g %12.4g  IDL: %s" % (var,val_ip,val_h,testdata[var.name]))
        else:
            print("%s %12.4g %12.4g  IDL: %s" % (var,val_ip,val_h,"---"))
#          print(var,np.squeeze(np.array(ctipe[var](positions[0],positions[2],positions[3]))))
#            except:
#                val_ip=np.squeeze(np.array(ctipe[var](pos_ip[0],pos_ip[1],pos_ip[2],pos_ip[3])))
#                val_h=np.squeeze(np.array(ctipe[var](positions[0],ilev,positions[2],positions[3])))
#                print(var,val_ip,val_h)
##                print(var,np.squeeze(np.array(ctipe[var]([positions[0],positions[1],positions[2],positions[3]]))))

[1426810500.0, 5.5, 10, 50]
[1426810500.0, 126200.0, 10, 50]
[1426810500.0, array(7.08655089), 10, 50]
rho(t, ilev, lat, lon)    1.491e-07     1.64e-08  IDL: 1.49067e-07  1.64051e-08
H(t, ilev, lat, lon)    1.052e+05     1.25e+05  IDL: 105.230      125.000
T(t, ilev, lat, lon)        298.4        430.8  IDL: 298.432      430.817
Rmt(t, ilev, lat, lon)        26.34        23.91  IDL: 26.3395      23.9059
N_e(t, ilev, lat, lon)     1.07e+08    5.931e+08  IDL: 1.06982e+08  5.90486e+08
N_Oplus(t, ilev, lat, lon)        3.414    4.988e+06  IDL: 3.41415  4.96166e+06
N_Hplus(t, ilev, lat, lon)            0        32.59  IDL: 0.00000      32.4158
T_i(t, ilev, lat, lon)          nan          nan  IDL: ---
T_e(t, ilev, lat, lon)          nan          nan  IDL: ---
Vn_lat(t, ilev, lat, lon)      0.06011       -43.47  IDL: -0.0601084      43.4968
Vn_lon(t, ilev, lat, lon)       -23.27       -3.148  IDL: -23.2656     -3.15513
Vn_H(t, ilev, lat, lon)     -0.04343     -0.08185  IDL: ---
T_n(t, ilev, 